In [1]:
# import json
# import requests

# solditems = requests.get('http://seppe.net/aa/assignment2/dataset.json')
# data = solditems.json()
# with open('dataset.json', 'w') as f:
#     json.dump(data, f)

In [2]:
import pandas as pd

In [3]:
def combine(row):
    return [{image_id: row["CUISINES"]} for image_id in row["IMAGE_IDS"]]

df = pd.read_json("dataset.json")
df = df[["cuisines", "more_details"]]


french = ["modern french", "french contemporary", "french"]

df["cuisines"] = df.cuisines.apply(lambda cuisines: [cuisine["label"] for cuisine in cuisines])
df["cuisines"] = df["cuisines"].apply(lambda cuisines: " ".join(str(cuisine).lower() for cuisine in cuisines))
df["cuisines"] = df.apply(lambda x: "french" if x["cuisines"] in french else x["cuisines"], axis=1)

df_count = df.groupby(df.cuisines, as_index=False).size().sort_values(by=["size"], ascending=False)[:20]

df["more_details"] = df.more_details.apply(lambda images: [image["image_id"] for image in images["full_images"]])
df.rename(columns={"cuisines": "CUISINES", "more_details": "IMAGE_IDS"}, inplace=True)

ids_and_cuisines = [item for sublist in list(df.apply(combine, axis=1)) for item in sublist]

df_images = pd.DataFrame(
    zip([list(image.keys())[0] for image in ids_and_cuisines], 
       [list(image.values())[0] for image in ids_and_cuisines]), 
    columns=["IMAGE_ID", "CUISINES"]
)

In [4]:
df_local_paths = pd.read_parquet("dataset_paths_annotated.parquet")
df_local_paths["IMAGE_ID"] = df_local_paths.LOCAL_PATH.apply(lambda x: x.split("/")[-1].split(".")[0])

df_merged = df_local_paths.merge(df_images, on="IMAGE_ID")
df_merged_filtered = df_merged[df_merged.CUISINES.isin(list(df_count.cuisines))]
df_merged_filtered = df_merged_filtered[df_merged_filtered.LABEL=="True"]
df_merged_filtered.reset_index(drop=True, inplace=True)
df_merged_filtered.to_parquet("dataset_merged.parquet")

In [5]:
# df_merged_filtered.groupby(df_merged_filtered.CUISINES, as_index=False).size().sort_values(by=["size"], ascending=False)